In [ ]:
import random
from datetime import datetime, timedelta

# Generate order IDs
order_ids = [f'2024-{str(i).zfill(2)}' for i in range(1, 51)]

# Initialize lists for storing generated data
orders = []

# Function to get the start date of a specific week in a given year
def get_week_start(year, week):
    # Calculate the start of the week (Monday)
    return datetime.strptime(f'{year}-W{str(week).zfill(2)}-1', "%Y-W%W-%w")

# Randomly generate order dates, delivery dates, and factories
for order_id in order_ids:
    # Extract the week number from the order ID
    order_week = int(order_id.split('-')[1])
    
    # Calculate the start date of the order ID week
    order_week_start = get_week_start(2024, order_week)
    
    # Generate a random order date 1 or 2 weeks before the order week
    order_date = order_week_start - timedelta(weeks=random.choice([1, 2]), days=random.randint(0, 6))
    
    # Generate a random delivery date 4 or 5 weeks after the order week
    delivery_date = order_week_start + timedelta(weeks=random.choice([4, 5]), days=random.randint(0, 6))
    
    # Randomly assign a factory
    factory = random.choice(['China', 'Vietnam'])
    
    # Append the generated data to the list
    orders.append({
        'Order ID': order_id,
        'Order Date': order_date.strftime('%Y-%m-%d'),
        'Delivery Date': delivery_date.strftime('%Y-%m-%d'),
        'Factory': factory
    })

# Display the results
for order in orders:
    print(order)


In [ ]:
def get_sales()->list[str]:
    '''
    docString
    parameter:
    return:
        傳出所有的城市名稱
    '''
    conn = sqlite3.connect("salesorders.db")
    with conn:
        # Create a cursor object to execute SQL commands
        cursor = conn.cursor()
        # SQL query to select unique sitenames from records table
        sql = '''
        SELECT DISTINCT sales_names
        FROM sales_orders
        '''
        # Execute the SQL query
        cursor.execute(sql)
        # Get all results and extract first item from each row into a list
        sales_names = [items[0] for items in cursor.fetchall()]
    
    # Return the list of unique sitenames
    
    return sales_names
print(sales_names)




In [ ]:
# Function to save data to SQLite database
def save_to_sqlite(df, db_filename='sales_order1.db'):
    conn = sqlite3.connect(db_filename)
    table_name = 'sales_orders1'
    
    # Create the table
    conn.execute(f"DROP TABLE IF EXISTS {table_name}")
    create_table_query = f'''
    CREATE TABLE {table_name} (
        
        "Sales ID" TEXT,
        "Sales Name" TEXT,
        "Customer ID" TEXT,
        "Order ID" TEXT,
        "Yield Rate" REAL,
        "Thru_put" REAL,
        "Order Date" TEXT,
        "Delivery Date" TEXT,
        "Factory" TEXT
    )
    '''
    conn.execute(create_table_query)
    
    # Insert data into SQLite
    df.to_sql(table_name, conn, if_exists='append', index=False)
    conn.commit()
    conn.close()
    print(f"Data has been successfully imported into '{db_filename}' database, table '{table_name}'.")

In [3]:
import sqlite3

# Function to save data to SQLite database one by one
def save_to_sqlite(df, db_filename='sales_order1.db'):
    print("Saving data to SQLite database record by record...")
    conn = sqlite3.connect(db_filename)
    table_name = 'sales_orders1'

    # Create the table with an auto-incrementing 'id' field
    create_table_query = f'''
    CREATE TABLE IF NOT EXISTS {table_name} (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        "Sales ID" TEXT,
        "Sales Name" TEXT,
        "Customer ID" TEXT,
        "Order ID" TEXT,
        "Yield Rate" REAL,
        "Thru_put" REAL,
        "Order Date" TEXT,
        "Delivery Date" TEXT,
        "Factory" TEXT
    )
    '''
    
    conn.execute(create_table_query)

    # Insert data into SQLite record by record
    cursor = conn.cursor()
    item = 1  # Initialize item counter
    
    for _, row in df.iterrows():
        sales_id = row['Sales ID']
        sales_name = row['Sales Name']
        customer_id = row['Customer ID']
        order_id = row['Order ID']
        yield_rate = float(row['Yield Rate']) if row['Yield Rate'] != '' else 0.0
        thru_put = float(row['Thru_put']) if row['Thru_put'] != '' else 0.0
        order_date = row['Order Date']
        delivery_date = row['Delivery Date']
        factory = row['Factory']

        # Insert each record one by one
        sql = f'''
        INSERT INTO {table_name} 
        ("Sales ID", "Sales Name", "Customer ID", "Order ID", "Yield Rate", "Thru_put", "Order Date", "Delivery Date", "Factory")
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        '''
        
        cursor.execute(sql, (sales_id, sales_name, customer_id, order_id, yield_rate, thru_put, order_date, delivery_date, factory))
        print(f"Inserted item {item}: {sales_id}, {sales_name}, {customer_id}, {order_id}")
        item += 1
    
    conn.commit()
    conn.close()
    print(f"Data has been successfully imported into '{db_filename}' database, table '{table_name}'.")

# Example usage
import pandas as pd

# Assuming 'df' is your DataFrame loaded from a CSV
df = pd.read_csv('sales_data.csv')  # Replace 'sales_data.csv' with your actual file
save_to_sqlite(df)


FileNotFoundError: [Errno 2] No such file or directory: 'sales_data.csv'